# NEURAL MACHINE TRANSLATION - GRU

## Required Module & Config files

In [1]:
import src.GRU as gruNMT
from src.Tokenizer import Corpus, LangData, dataLoader
from src.utils import load_config, get_device, train_model, torch_bleu_score
from src.Normalizer import preprocess_data
from torch.nn import CrossEntropyLoss
from src.Translator import  Translator
from torch.optim import NAdam
import numpy as np

In [2]:
# Loading config file
config = load_config()
# Get device : GPU/MPS Back-End/CPU
device = get_device()
print(f"Using device: {device}")

Using device: mps


## Data Preprocessing

In [3]:
# # TRAIN_DATA
# preprocess_data(config.TRAIN_RAW, config.TRAIN_DATA, config.TRAIN_SOURCE, "english")
# preprocess_data(config.TRAIN_RAW, config.TRAIN_DATA, config.TRAIN_TARGET, "afrikaans")

# # VAL_DATA
# preprocess_data(config.VAL_RAW, config.VAL_DATA, config.VAL_SOURCE, "english")
# preprocess_data(config.VAL_RAW, config.VAL_DATA, config.VAL_TARGET, "afrikaans")

## Load the dataset

In [4]:
# Encoder-Source
english_data = Corpus(f"{config.TRAIN_DATA}/english.txt", "English")
afrikaans_data = Corpus(f"{config.TRAIN_DATA}/afrikaans.txt", "Afrikaans")

## Set Hyperparameters

In [5]:
# Encoder - source
IN_ENCODER = english_data.vocab_size
ENCODER_EMB = 256

# Decoder - target
IN_DECODER = afrikaans_data.vocab_size
OUT_DECODER = afrikaans_data.vocab_size
DECODER_EMB = 256

# Shared
HIDDEN_SIZE = 1024
NUM_LAYERS = 2

LR = 1e-3
BATCH_SIZE = 128

## Set the model

In [6]:
encoder_net = gruNMT.Encoder(IN_ENCODER, ENCODER_EMB, HIDDEN_SIZE, NUM_LAYERS).to(device)
decoder_net = gruNMT.Decoder(IN_DECODER, DECODER_EMB, HIDDEN_SIZE, NUM_LAYERS).to(device)
model = gruNMT.GRU_NMT(encoder_net, decoder_net, OUT_DECODER)

In [9]:
train_data = LangData(english_data, afrikaans_data)
train_loader = dataLoader(train_data, BATCH_SIZE)

pad_idx = afrikaans_data.stoi['<pad>']
criterion = CrossEntropyLoss(ignore_index=0)

optimizer = NAdam(model.parameters(), LR)
translator = Translator(model, english_data, afrikaans_data, device)

In [10]:
# Data used for follow-up durring training
mytext = "<sos> the classifier makes <num> correct positive predictions and <num> <com> <num> correct negative predictions <eos>"
ground = "<sos> die klassifiseerder maak <num> korrekte positiewe voorspellings en <num> <com> <num> korrekte negatiewe voorspellings <eos>"

predicted = translator.translate_sentence(mytext, method="beam")
bleu = torch_bleu_score([predicted], [ground])
print(f"Pred: {predicted}")
print(f"Refe: {ground}")
print(f"BLEU: {bleu.item()}")

Pred: <sos> werknemers vrae vrae vrae vrae beeld sirkul sirkul a hul hul hul hul hul vyf vyf vyf vyf al filter
Refe: <sos> die klassifiseerder maak <num> korrekte positiewe voorspellings en <num> <com> <num> korrekte negatiewe voorspellings <eos>
BLEU: 0.0


## Train the data

In [12]:
EPOCHS = 20
params = {
    "model": model,
    "train_loader": train_loader,
    "optimizer": optimizer,
    "criterion": criterion,
    "device": device,
    "epochs": EPOCHS,
    "source_test": mytext,
    "target_test": ground,
	"translator":translator
}

train_model(**params)

Epoch 1/20: 100%|██████████| 4/4 [00:01<00:00,  2.03batch/s, loss=0.380]


Predicted: <sos> die sein <ltx> word geanaliseer deur <apo>n analoog na syfer omsetter <opn> adc <cld> teen <apo>n monsterfrekwensie van <ltx> khz
Reference: <sos> die klassifiseerder maak <num> korrekte positiewe voorspellings en <num> <com> <num> korrekte negatiewe voorspellings <eos>
BLEU Score: 0.0


Epoch 2/20: 100%|██████████| 4/4 [00:02<00:00,  1.92batch/s, loss=0.355]


Predicted: <sos> die diskrete sein word opgemonster opgemonster met <apo>n faktor <num> opgemonster <opn> upsample <cld> <com> gefilter deur <apo>n banddeurlaatfilter <opn>
Reference: <sos> die klassifiseerder maak <num> korrekte positiewe voorspellings en <num> <com> <num> korrekte negatiewe voorspellings <eos>
BLEU Score: 0.0


Epoch 3/20: 100%|██████████| 4/4 [00:01<00:00,  2.30batch/s, loss=0.288]


...

Predicted: <sos> die klassifiseerder maak <num> korrekte positiewe voorspellings en <num> <com> <num> korrekte negatiewe voorspellings <eos>
Reference: <sos> die klassifiseerder maak <num> korrekte positiewe voorspellings en <num> <com> <num> korrekte negatiewe voorspellings <eos>
BLEU Score: 1.0


Epoch 19/20: 100%|██████████| 4/4 [00:02<00:00,  1.94batch/s, loss=0.225]


Predicted: <sos> die klassifiseerder maak <num> korrekte positiewe voorspellings en <num> <com> <num> korrekte negatiewe voorspellings <eos>
Reference: <sos> die klassifiseerder maak <num> korrekte positiewe voorspellings en <num> <com> <num> korrekte negatiewe voorspellings <eos>
BLEU Score: 1.0


Epoch 20/20: 100%|██████████| 4/4 [00:01<00:00,  2.22batch/s, loss=0.199]

Predicted: <sos> die klassifiseerder maak <num> korrekte positiewe voorspellings en <num> <com> <num> korrekte negatiewe voorspellings <eos>
Reference: <sos> die klassifiseerder maak <num> korrekte positiewe voorspellings en <num> <com> <num> korrekte negatiewe voorspellings <eos>
BLEU Score: 1.0


## EVALUATE

In [52]:
EN_STR = [[' '.join(sent)] for sent in english_data.data_str]
AF_STR = [[' '.join(sent)] for sent in afrikaans_data.data_str]
TRANSLATED = [[translate_sentece(model, sent[0], english_data, afrikaans_data, device)] for sent in EN_STR]

In [53]:
BLEU_SCORE = [torch_bleu_score(a,b) for a, b in zip(TRANSLATED, AF_STR)]
print(f"Mean BLEU TRAIN {np.mean(BLEU_SCORE)}")

Mean BLEU TRAIN 0.8744739294052124


In [54]:
with open(f"{config.VAL_DATA}/english.txt") as data:
	english_test = data.read().strip().split("\n")
with open(f"{config.VAL_DATA}/afrikaans.txt") as data:
	afrikaans_test = data.read().strip().split("\n")
AF_TEST = [[sent] for sent in afrikaans_test]

In [55]:
TRANSLATED_VAL = [[translate_sentece(model, sent, english_data, afrikaans_data, device)] for sent in english_test]

In [56]:
BLEU_VAL= [torch_bleu_score(a,b) for a, b in zip(TRANSLATED_VAL, AF_TEST)]
print(f"Mean BLEU VAL {np.mean(BLEU_VAL)}")

Mean BLEU VAL 0.02052861452102661


In [61]:
data_eng = [sent.strip().split() for sent in english_test]

In [64]:
data_eng1 = []
for sent in data_eng:
	 for word in sent:
		 data_eng1.append(word if word in english_data.stoi else '<unk>')

In [66]:
from collections import Counter

In [67]:
A = Counter(data_eng1)

In [68]:
A

Counter({'<unk>': 184140105,
         'e': 50591803,
         's': 40394087,
         't': 31692371,
         'a': 24609015,
         'n': 23570834,
         'i': 21799972,
         'c': 18197177,
         'd': 13037243,
         'b': 7052933,
         'y': 4579824,
         'z': 366384,
         'the': 236,
         '<eos>': 194,
         '<sos>': 182,
         '<num>': 100,
         'of': 82,
         'is': 58,
         'and': 55,
         'to': 54,
         '<com>': 43,
         'what': 41,
         'for': 33,
         'system': 25,
         'describe': 25,
         '<opn>': 24,
         '<cld>': 24,
         'in': 23,
         'an': 22,
         'below': 19,
         'this': 17,
         'on': 16,
         'between': 15,
         'be': 15,
         'that': 15,
         'are': 15,
         'each': 14,
         'code': 14,
         'with': 14,
         '<ltx>': 13,
         'used': 13,
         'your': 12,
         'diagram': 12,
         'design': 11,
         'which': 10,
         